In [ ]:
!pip install datasets
!pip install evaluate
!pip install rouge_score
!pip install accelerate -U
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c665f1c57499d84e66b74d5d9b0ecc00417ce964b9dd7228cab764e85df9734b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
%cd /content/drive/MyDrive/BBC News Summary

/content/drive/MyDrive/BBC News Summary


In [ ]:
!pwd

/content/drive/MyDrive/BBC News Summary


In [ ]:
# Importing libraries
import os
import glob
import numpy as np
import pandas as pd

In [ ]:
#read news dataset
filepath = "/content/drive/MyDrive/Colab Notebooks/BBC_news_dataset.csv"
df = pd.read_csv(filepath)
df.head()

,Articles,Summaries,Categories
0,Ad sales boost Time Warner profit\n\nQuarterly...,TimeWarner said fourth quarter sales rose 2% t...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...,business
3,High fuel prices hit BA's profits\n\nBritish A...,"Rod Eddington, BA's chief executive, said the ...",business
4,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...,business


In [ ]:
# Getting article and summary length
df['Article Length'] = df["Articles"].apply(lambda x: len(x.split()))
df['Summary Length'] = df["Summaries"].apply(lambda x: len(x.split()))

In [ ]:
#shuffling dataframe before splitting dataset, so that
df = df.sample(frac=1)

In [ ]:
df.columns

Index(['Articles', 'Summaries', 'Categories', 'Article Length',
       'Summary Length'],
      dtype='object')

In [ ]:
df.shape

(2225, 5)

In [ ]:
# splitting dataset into train, test and validation dataset
# Train : Test : Val = 70:20:10
df2 = df[['Articles', 'Summaries']]

train_data = df2[:int(0.7*df.shape[0])]
val_data = df2[int(0.7*df.shape[0]):int(0.8*df.shape[0])]
test_data = df2[int(0.8*df.shape[0]):]

In [ ]:
train_data.shape, test_data.shape, val_data.shape

((1557, 2), (445, 2), (223, 2))

In [ ]:
# storing these dataset to csv file for future direct use

# train_data.to_csv("train.csv", index=False)
# val_data.to_csv("val.csv", index=False)
# test_data.to_csv("test.csv", index=False)

# train_data = pd.read_csv("train.csv")
# val_data = pd.read_csv("val.csv")
# test_data = pd.read_csv("test.csv")

In [ ]:
from transformers import AutoTokenizer

base_model = "t5-small"
# base_model = 'it5/it5-base-news-summarization'

tokenizer = AutoTokenizer.from_pretrained(base_model)

# adding prefix for T5 models
prefix = 'summarize: '

input_max_length = 512
target_max_length = 128

# tokenize the dataset
def encode_batch(examples):
    text_column = 'Articles'
    summary_column = 'Summaries'
    padding = "max_length"

    # convert to lists of strings
    inputs, targets = [], []
    for i in range(len(examples[text_column])):
        if examples[text_column][i] and examples[summary_column][i]:
            inputs.append(prefix + examples[text_column][i])
            targets.append(examples[summary_column][i])


    model_inputs = tokenizer(inputs, max_length=input_max_length, padding=padding, truncation=True)
    labels = tokenizer(targets, max_length=target_max_length, padding=padding, truncation=True)

    # rename to labels for training
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
from datasets import load_dataset, Dataset

## function to tokenize the data in batch
def load_dataset(data):
  dataset_obj = Dataset.from_pandas(data)
  tokenized_data = dataset_obj.map(
        encode_batch,
        batched=True,
        remove_columns=dataset_obj.column_names,
    )
  tokenized_data.set_format(type="torch", columns=["input_ids", "labels"])
  return tokenized_data

In [ ]:
train_tokenized_data = load_dataset(train_data)
test_tokenized_data = load_dataset(test_data)
val_tokenized_data = load_dataset(val_data)

Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

In [ ]:
train_tokenized_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1557
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=base_model)

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
# access_token = "hf_mveNqnsCmDYZpDBhHdGxzG...."
model = AutoModelForSeq2SeqLM.from_pretrained(base_model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./my_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=False,
    save_total_limit = 2,
    save_strategy = "epoch",
    load_best_model_at_end=True,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_data,
    eval_dataset=val_tokenized_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()
# import torch
# torch.cuda.empty_cache()

In [ ]:
# train on complete data
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.752737,0.177500,0.118700,0.158100,0.157900,18.062800
2,1.110100,0.682200,0.178500,0.125200,0.162100,0.162600,17.466400
3,0.739200,0.653441,0.193100,0.138500,0.177100,0.177300,18.574000
4,0.676600,0.635428,0.198500,0.145600,0.184000,0.184400,18.914800
5,0.676600,0.624578,0.200100,0.148600,0.185900,0.185700,18.914800
6,0.640800,0.617297,0.207200,0.158400,0.194100,0.194000,19.000000
7,0.622500,0.612026,0.209500,0.160400,0.195900,0.196200,19.000000
8,0.617400,0.609243,0.211100,0.164000,0.197900,0.198200,19.000000
9,0.598800,0.607350,0.215600,0.169200,0.202600,0.202800,19.000000
10,0.598800,0.606775,0.215900,0.169300,0.202900,0.203100,19.000000


TrainOutput(global_step=3900, training_loss=0.7036775911771335, metrics={'train_runtime': 1010.486, 'train_samples_per_second': 15.408, 'train_steps_per_second': 3.86, 'total_flos': 2107271848919040.0, 'train_loss': 0.7036775911771335, 'epoch': 10.0})

In [ ]:
# evaluating best model on validation dataset
trainer.evaluate()

{'eval_loss': 0.6067754030227661,
 'eval_rouge1': 0.2159,
 'eval_rouge2': 0.1693,
 'eval_rougeL': 0.2029,
 'eval_rougeLsum': 0.2031,
 'eval_gen_len': 19.0,
 'eval_runtime': 19.8844,
 'eval_samples_per_second': 11.215,
 'eval_steps_per_second': 2.816,
 'epoch': 10.0}

In [ ]:
trainer.evaluate(eval_dataset = test_tokenized_data)

{'eval_loss': 0.5863471627235413,
 'eval_rouge1': 0.2141,
 'eval_rouge2': 0.1645,
 'eval_rougeL': 0.2007,
 'eval_rougeLsum': 0.2006,
 'eval_gen_len': 19.0,
 'eval_runtime': 48.0938,
 'eval_samples_per_second': 9.253,
 'eval_steps_per_second': 2.329,
 'epoch': 10.0}

In [ ]:
# saving finetuned model at
model_stored_at = "./finetuned_model"
trainer.save_model(model_stored_at)
model.save_pretrained (model_stored_at)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# function for inferencing without using pipeline
def summarize_text(text):
  text = "summarize: " + text
  tokenizer = AutoTokenizer.from_pretrained(model_stored_at)
  inputs = tokenizer(text, return_tensors="pt").input_ids

  model = AutoModelForSeq2SeqLM.from_pretrained(model_stored_at)
  outputs = model.generate(inputs, max_new_tokens=128, do_sample=False)

  return tokenizer.decode(outputs[0], skip_special_tokens=True)

**Inferencing on Finetuned model**

In [ ]:
text = """The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs.
It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers
 and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and
  corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."""
summarize_text(text)

"The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs.It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share.It'll ask the ultra-wealthy and corporations to pay their fair share."